In [1]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')
df.head(20)

,Unnamed: 0,file,message,subject,text,date,from,to,emotion_scores
0,240475,216050,239080,132210,6562 12510 5447 12122 14368 14125 16224 4597 1...,0,17917,39435,[0. 0.09375 0. 0.03125 0.125 0. ...
1,231468,208137,177449,121217,3833 6660 13658 446 16658 7193 6660 3958 12375...,0,14350,13150,[0.02777778 0.05555556 0.05555556 0.05555556 0...
2,231581,208236,276335,121217,10861 4628 10928 17400 7741 5489 18357 15348 1...,0,1422,13448,[0.2 0. 0. 0.2 0. 0.2 0.2 0.2 0. 0. ]
3,231582,208237,116126,121217,3306 13193 16689 2184 11333 6726 5981 7460 813...,0,1422,13447,[0.05533597 0.08498024 0.02371542 0.06719368 0...
4,231584,208239,321587,121217,6145 3023 11333 6480 3546 3776 4502 1024,0,1422,13445,[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
5,231585,208240,432763,121217,7370 15999 10732 12100 446 446 1050 11333 4502...,0,1422,13444,[0. 0. 0. 0. 0. 0. 0.5 0. 0. 0.5]
6,231589,208243,330564,121217,11096 11333 6458 8279 6018 4243 1029 11333 797...,0,1422,13443,[0.05263158 0.0877193 0.01754386 0.07017544 0...
7,231590,208244,415039,121217,6987 6044 8603 7643 7469 5470 8443 11333 5780 ...,0,1422,13442,[0. 0. 0.25 0. 0. 0.25 0. 0. 0. ...
8,231808,208440,225425,121217,3399 5112 9223 7591 5085 2478 9758 13230 2266 ...,0,12482,13678,[0. 0.11111111 0. 0.03703704 0...
9,90767,83541,467279,56716,4669 3082 3693 4755 4511 4285 2745 3329 170 29...,0,4463,31312,[0.06666667 0.06666667 0.06666667 0.06666667 0...


In [2]:
import numpy as np
import pandas as pd

df = pd.read_csv('../september/result/september.csv')


array = np.zeros((len(df), 10), dtype=int)

for i in range(len(df)):
    text_value = df.loc[i, 'text']
    if isinstance(text_value, str):  
        text_values = list(map(int, text_value.split()))
        length = min(len(text_values), 10)  
        array[i, :length] = text_values[:length]

# with open('output_array.txt', 'w') as f:
#     for row in array:
#         f.write(' '.join(map(str, row)) + '\n')  



In [3]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')

vector_accuracy  = [0] * len(df)

for i in range(len(df)):
    text_values = df.loc[i, 'emotion_scores'].strip('[]').split()  

    if text_values:
        first_emotion_score = float(text_values[6])  
        if first_emotion_score < 0.3:
            vector_accuracy [i] = 0
        else:
            vector_accuracy [i] = 1

# with open('vector_accuracy.txt', 'w') as f:
#     for value in vector_accuracy:
#         f.write(str(value) + '\n')  


In [4]:
labels = []
vector_w = [1000] * 10


for nums in array:
    temp = np.dot(nums, vector_w)
    if temp >= 0:
        labels.append(1)
    else:
        labels.append(0)

# with open('label.txt', 'w') as f:
#     for value in labels:
#         f.write(str(value) + '\n')  

In [5]:
vector_errors = []

for i in range(len(array)):
    vector_errors.append(vector_accuracy[i] - labels[i])

print("vector_errors: ", vector_errors)

# with open('vector_errors.txt', 'w') as f:
#     for value in vector_errors:
#         f.write(str(value) + '\n')  


vector_errors:  [0, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, -1, 0, -1, 0, 0, -1, -1, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [6]:
results = []
for epoch in range(100):

    delta = [0] * 10

    for i in range(len(array)):
        for j in range(len(array[0])):
            delta[j] += vector_errors[i] * 0.1 * array[i][j]

    for i in range(10):
        vector_w[i] = vector_w[i] + delta[i]

    labels = []

    for nums in array:
        temp = np.dot(nums, vector_w)
        if temp >= 0:
            labels.append(1)
        else:
            labels.append(0)

    vector_errors = []

  
    for i in range(len(array)):
        vector_errors.append(vector_accuracy[i] - labels[i])

    # with open('vector_errors.txt', 'w') as f:
    #     for value in vector_errors:
    #         f.write(str(value) + '\n')  

    with open('epoh1.txt', 'a') as f:
        TP = sum((labels[i] == 1 and vector_accuracy[i] == 1) for i in range(len(labels)))
        TN = sum((labels[i] == 0 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FP = sum((labels[i] == 1 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FN = sum((labels[i] == 0 and vector_accuracy[i] == 1) for i in range(len(labels)))

        # Расчет метрик
        N = len(array)  # Общее количество наблюдений
        accuracy = (TP + TN) / N 
        precision = TP / (TP + FP) 
        recall = TP / (TP + FN) 
        Fscore = TP / (TP + FN + FP) 

        # Запись результатов в файл
        f.write(f'Эпоха {epoch} ------------------------------------\n')
        f.write(f'accuracy = {accuracy:.4f}\n')
        f.write(f'precision = {precision:.4f}\n')
        f.write(f'recall = {recall:.4f}\n')
        f.write(f'Fscore = {Fscore:.4f}\n\n')

    results.append({
        'Epoch': epoch,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Fscore': Fscore
    })

results_df = pd.DataFrame(results)
results_df.to_csv('./ignore/results_analysis.csv', index=False)


In [7]:
import numpy as np
import pandas as pd

df = pd.read_csv('../september/result/september.csv')


array = np.zeros((len(df), 10), dtype=int)

for i in range(len(df)):
    text_value = df.loc[i, 'text']
    if isinstance(text_value, str):  
        text_values = list(map(int, text_value.split()))
        length = min(len(text_values), 10)  
        array[i, :length] = text_values[:length]

# with open('output_array.txt', 'w') as f:
#     for row in array:
#         f.write(' '.join(map(str, row)) + '\n')  



In [8]:
import pandas as pd

df = pd.read_csv('../september/result/september.csv')

vector_accuracy  = [0] * len(df)

for i in range(len(df)):
    text_values = df.loc[i, 'emotion_scores'].strip('[]').split()  

    if text_values:
        first_emotion_score = float(text_values[6])  
        if first_emotion_score < 0.3:
            vector_accuracy [i] = 0
        else:
            vector_accuracy [i] = 1

# with open('vector_accuracy.txt', 'w') as f:
#     for value in vector_accuracy:
#         f.write(str(value) + '\n')  


In [9]:
import numpy as np

labels = []
vector_w = [1000] * 10


for nums in array:
    temp = np.dot(nums, vector_w)
    labels.append(round(1 / (1 + np.exp(-temp))))

# with open('label.txt', 'w') as f:
#     for value in labels:
#         f.write(str(value) + '\n')  

In [10]:
vector_errors = []

for i in range(len(array)):
    vector_errors.append(vector_accuracy[i] - labels[i])

print("vector_errors: ", vector_errors)

# with open('vector_errors.txt', 'w') as f:
#     for value in vector_errors:
#         f.write(str(value) + '\n')  


vector_errors:  [0, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, -1, 0, -1, 0, 0, -1, -1, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, 0, 0, -1, 0, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, -1, -1, -1, -1, 0, -1, 0, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, 0, 0, 0, -1, -1, 0, -1, -1, 0, -1, -1, 0, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, 0, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,

In [11]:
results = []
for epoch in range(100):

    delta = [0] * 10
    delta_w = 0

    for i in range(len(array)):
        for j in range(len(array[0])):
            delta[j] += vector_errors[i] * 0.1 * array[i][j]
            delta_w += vector_errors[i] * 0.1 

    for i in range(10):
        vector_w[i] = vector_w[i] + delta[i]

    labels = []

    for nums in array:
        temp = np.dot(nums, vector_w)
        labels.append(round(1 / (1 + np.exp(-temp))))

    vector_errors = []
  
    for i in range(len(array)):
        vector_errors.append(vector_accuracy[i] - labels[i])

    # with open('vector_errors.txt', 'w') as f:
    #     for value in vector_errors:
    #         f.write(str(value) + '\n')  

    with open('epoh2.txt', 'a') as f:
        TP = sum((labels[i] == 1 and vector_accuracy[i] == 1) for i in range(len(labels)))
        TN = sum((labels[i] == 0 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FP = sum((labels[i] == 1 and vector_accuracy[i] == 0) for i in range(len(labels)))
        FN = sum((labels[i] == 0 and vector_accuracy[i] == 1) for i in range(len(labels)))

        # Расчет метрик
        N = len(array)  # Общее количество наблюдений
        accuracy = (TP + TN) / N 
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        Fscore = TP / (TP + FN + FP) if (TP + FN + FP) > 0 else 0

        # Запись результатов в файл
        f.write(f'Эпоха {epoch} ------------------------------------\n')
        f.write(f'accuracy = {accuracy:.4f}\n')
        f.write(f'precision = {precision:.4f}\n')
        f.write(f'recall = {recall:.4f}\n')
        f.write(f'Fscore = {Fscore:.4f}\n\n')

    results.append({
        'Epoch': epoch,
        'Accuracy': accuracy,
        'Precision': precision,
        'Recall': recall,
        'Fscore': Fscore
    })

results_df = pd.DataFrame(results)
results_df.to_csv('./ignore/results_analysis_sigmoida.csv', index=False)


/var/folders/4z/x3rxm6xs5c5_01z63xh1mdhm0000gn/T/ipykernel_41414/1435012619.py:19: RuntimeWarning: overflow encountered in exp
  labels.append(round(1 / (1 + np.exp(-temp))))


In [12]:
import pandas as pd

# Загрузка данных из CSV файлов
file1 = './ignore/results_analysis.csv'
file2 = './ignore/results_analysis_sigmoida.csv'

data1 = pd.read_csv(file1)
data2 = pd.read_csv(file2)

# Вычисление средних значений метрик
metrics1 = data1[['Accuracy', 'Precision', 'Recall', 'Fscore']].mean()
metrics2 = data2[['Accuracy', 'Precision', 'Recall', 'Fscore']].mean()

# Вычисление стандартного отклонения метрик
std_metrics1 = data1[['Accuracy', 'Precision', 'Recall', 'Fscore']].std()
std_metrics2 = data2[['Accuracy', 'Precision', 'Recall', 'Fscore']].std()

# Вывод результатов
print("Средние значения метрик для первого файла:")
print(metrics1)
print("Стандартные отклонения метрик для первого файла:")
print(std_metrics1)

print("\nСредние значения метрик для второго файла:")
print(metrics2)
print("Стандартные отклонения метрик для второго файла:")
print(std_metrics2)

# Сравнение метрик
for metric in metrics1.index:
    if metrics1[metric] > metrics2[metric]:
        print(f"{metric} больше при линейной классификации.")
    elif metrics1[metric] < metrics2[metric]:
        print(f"{metric} больше при сигмоидальной классификации.")
    else:
        print(f"{metric} одинаков при обоих классификациях.")


Средние значения метрик для первого файла:
Accuracy     0.632408
Precision    0.529407
Recall       0.263570
Fscore       0.092399
dtype: float64
Стандартные отклонения метрик для первого файла:
Accuracy     0.187772
Precision    0.155002
Recall       0.399063
Fscore       0.112457
dtype: float64

Средние значения метрик для второго файла:
Accuracy     0.632374
Precision    0.187426
Recall       0.263286
Fscore       0.092179
dtype: float64
Стандартные отклонения метрик для второго файла:
Accuracy     0.187772
Precision    0.220915
Recall       0.399063
Fscore       0.112543
dtype: float64
Accuracy больше при линейной классификации.
Precision больше при линейной классификации.
Recall больше при линейной классификации.
Fscore больше при линейной классификации.
